## Introduction
#### 데이터 분석의 대부분은 데이터를 불러오고, 정제하고, 변형하는 준비에 대부분의 시간이 소모된다. 
#### 이장에서는 결측치, 중복데이터, 문자열 데이터, 그리고 다른 분석적 데이터 변환에 대한 도구들을 다룬다. 

## 누락된 데이터 처리하기 
#### pandas객체의 모든 기술 통계는 누락된 데이터를 배제하고 처리한다.
#### pandas에서는 누락된 데이터를 실수값인 NaN으로 취급한다. 이는 누락된 값을 쉽게 찾을 수 있도록 하는 역할을 한다.

In [1]:
import pandas as pd
import numpy as np

float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [2]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
string_data = pd.Series(['aardvak', 'artichoke', np.nan, 'avocado'])
string_data

0      aardvak
1    artichoke
2          NaN
3      avocado
dtype: object

In [5]:
string_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

In [6]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

#### NaN은 데이터가 존재하지 않거나, 존재하더라도 데이터를 수집하는 과정에서 검출되지 않았음을 의미한다. 
#### 데이터를 정제하는 과정에서 결측치 자체를 데이터 수집과정에서의 실수나 결측치로 인한 잠재적 편향을 찾아내는 수단으로 인식해야 한다.
#### 표 7-1 NA 처리 메소드 참조

In [7]:
string_data[0] = None

string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 누락된 데이터 골라내기 

In [8]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [7]:
data[data.notnull()] #dropna와 동일한 결과

0    1.0
2    3.5
4    7.0
dtype: float64

In [10]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [8]:
#DataFRame 객체의 경우 좀 복잡하다. 
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                   [NA, NA, NA], [NA, 6.5, 3.]])

cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [9]:
cleaned #DataFrame 객체에서 dropna는 기본적으로 NA 값을 하나라도 포함하고 있는 row를 제거한다.

,0,1,2
0,1.0,6.5,3.0


In [10]:
data.dropna(how='all') #how='all' 옵션의 경우 모두 NA인 row만 제거한다.

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [11]:
data[4] = NA  #4번째 컬럼에 NaN추가
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [12]:
data[0]
#data.iloc[0]
#data.loc[0]

0    1.0
1    1.0
2    NaN
3    NaN
Name: 0, dtype: float64

In [13]:
data.dropna(axis=1, how='all') #컬럼을 제거하는 것도 동일하다. 옵션으로 axis=1을 넘겨준다.

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


#### DataFrame의 row를 제거하는 방법은 시계열에서 주로 사용된다. 몇개 이상의 값이 들어있는 row를 보고 싶으면 thresh 인자를 사용 

In [14]:
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA

df

,0,1,2
0,2.267959,NaN,NaN
1,-0.279387,NaN,NaN
2,-0.484572,NaN,-0.248683
3,-0.353412,NaN,1.984146
4,-1.282566,0.028845,0.439538
5,0.095093,0.382246,0.338562
6,1.179651,-0.059137,-1.685796


In [15]:
df.dropna()

,0,1,2
4,-1.282566,0.028845,0.439538
5,0.095093,0.382246,0.338562
6,1.179651,-0.059137,-1.685796


In [16]:
df.dropna(thresh=2) #결측치를 제외하고 최소 2개 이상인 데이터를 가진것만 것만  

,0,1,2
2,-0.484572,NaN,-0.248683
3,-0.353412,NaN,1.984146
4,-1.282566,0.028845,0.439538
5,0.095093,0.382246,0.338562
6,1.179651,-0.059137,-1.685796


In [17]:
df.dropna(axis=1, thresh=2)  

,0,1,2
0,2.267959,NaN,NaN
1,-0.279387,NaN,NaN
2,-0.484572,NaN,-0.248683
3,-0.353412,NaN,1.984146
4,-1.282566,0.028845,0.439538
5,0.095093,0.382246,0.338562
6,1.179651,-0.059137,-1.685796


### 결측치 채우기 
#### 누락한 값을 제외시키지 않고 데이터상의 '구멍'을 메우고 싶을때, fillna메소드를 사용한다. 

In [18]:
df

,0,1,2
0,2.267959,NaN,NaN
1,-0.279387,NaN,NaN
2,-0.484572,NaN,-0.248683
3,-0.353412,NaN,1.984146
4,-1.282566,0.028845,0.439538
5,0.095093,0.382246,0.338562
6,1.179651,-0.059137,-1.685796


In [19]:
df.fillna(0)

,0,1,2
0,2.267959,0.000000,0.000000
1,-0.279387,0.000000,0.000000
2,-0.484572,0.000000,-0.248683
3,-0.353412,0.000000,1.984146
4,-1.282566,0.028845,0.439538
5,0.095093,0.382246,0.338562
6,1.179651,-0.059137,-1.685796


In [20]:
df

,0,1,2
0,2.267959,NaN,NaN
1,-0.279387,NaN,NaN
2,-0.484572,NaN,-0.248683
3,-0.353412,NaN,1.984146
4,-1.282566,0.028845,0.439538
5,0.095093,0.382246,0.338562
6,1.179651,-0.059137,-1.685796


In [21]:
df.fillna({1:0.5, 2: 0}) #fillna에 사전값을 넘겨서 컬럼마다 다른값을 채울수도 있다. 

,0,1,2
0,2.267959,0.500000,0.000000
1,-0.279387,0.500000,0.000000
2,-0.484572,0.500000,-0.248683
3,-0.353412,0.500000,1.984146
4,-1.282566,0.028845,0.439538
5,0.095093,0.382246,0.338562
6,1.179651,-0.059137,-1.685796


In [22]:
df

,0,1,2
0,2.267959,NaN,NaN
1,-0.279387,NaN,NaN
2,-0.484572,NaN,-0.248683
3,-0.353412,NaN,1.984146
4,-1.282566,0.028845,0.439538
5,0.095093,0.382246,0.338562
6,1.179651,-0.059137,-1.685796


In [23]:
#fillna는 새로운 객체를 반환하지만 다음처럼 기존 객체를 변경할 수도 있다.
# 파이썬 마지막으로 실행된 결과값이 _라는 변수에 저장된다. 
# _없이 사용해도 무방 : df.fillna(0, inplace=True)
_ = df.fillna(0, inplace=True)  #inplace=True 옵션은 교체되는것을 참으로 처리하여 df값이 변경되도록 한다.

df

,0,1,2
0,2.267959,0.000000,0.000000
1,-0.279387,0.000000,0.000000
2,-0.484572,0.000000,-0.248683
3,-0.353412,0.000000,1.984146
4,-1.282566,0.028845,0.439538
5,0.095093,0.382246,0.338562
6,1.179651,-0.059137,-1.685796


In [24]:
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,-1.305739,2.435947,-0.477204
1,1.321581,0.506811,0.926032
2,-0.123949,NaN,1.645289
3,0.384124,NaN,-0.243372
4,0.442385,NaN,NaN
5,1.434449,NaN,NaN


In [25]:
obj = pd.Series(['blue', 'purple', 'yellow'], index=[0,2,4])
obj.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [26]:
df.fillna(method='ffill') #재색인에서 사용 가능한 보간 메소드는 fillna메소드에서도 사용 가능하다.

,0,1,2
0,-1.305739,2.435947,-0.477204
1,1.321581,0.506811,0.926032
2,-0.123949,0.506811,1.645289
3,0.384124,0.506811,-0.243372
4,0.442385,0.506811,-0.243372
5,1.434449,0.506811,-0.243372


In [27]:
#pad, ffill : Nan 값을 앞의 값으로 채운다
#bfill,backfill : Nan 값을 뒤의 값으로 채운다

df.fillna(method='bfill') #ffill 대신에 bfill 사용

,0,1,2
0,-1.305739,2.435947,-0.477204
1,1.321581,0.506811,0.926032
2,-0.123949,NaN,1.645289
3,0.384124,NaN,-0.243372
4,0.442385,NaN,NaN
5,1.434449,NaN,NaN


In [28]:
df.fillna(method='ffill', limit=2) #limit 옵션을 사용해서 2개까지만 보간

,0,1,2
0,-1.305739,2.435947,-0.477204
1,1.321581,0.506811,0.926032
2,-0.123949,0.506811,1.645289
3,0.384124,0.506811,-0.243372
4,0.442385,NaN,-0.243372
5,1.434449,NaN,-0.243372


In [29]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [30]:
data.fillna(data.mean()) #Series의 중간값을 전달

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

In [31]:
df.fillna(data.mean())#DataFrame의 중간값을 전달

,0,1,2
0,-1.305739,2.435947,-0.477204
1,1.321581,0.506811,0.926032
2,-0.123949,3.833333,1.645289
3,0.384124,3.833333,-0.243372
4,0.442385,3.833333,3.833333
5,1.434449,3.833333,3.833333


## 데이터 변형

### 중복 제거하기 
#### 여러가지 이유로 DataFrame에서 중복된 row를 발견할 수있다. 

In [53]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                    'k2': [1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [54]:
data.duplicated() #duplicated 메소드는 각 row가 중복인지 아닌지를 알려주는 Series를 반환한다. 

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [55]:
data.drop_duplicates() #drop_duplicates는 duplicated 배열이 False인 DataFrame을 반환한다.

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


#### 이 두 메소드는 기본적으로 모든 컬럼에 적용되며 중복을 찾아내가 위한 부분합을 따로 지정해줄수도 있다. 

In [56]:
#새로운 컬럼 하나를 추가하고 'k1'컬럼에 기반해 중복을 걸러낸다.
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [57]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [58]:
#duplicated와 drop_duplicates는 기본적으로 처음 발견된 값을 유지한다. 
data.drop_duplicates(['k1', 'k2']) 

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5


In [59]:
#duplicated와 drop_duplicates는 기본적으로 처음 발견된 값을 유지한다. 
#keep='last'옵션을 넘기면 마지막으로 발견된 값을 반환한다.
#data.drop_duplicates(['k1', 'k2']) 
data.drop_duplicates(['k1', 'k2'], keep='last') 

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 함수 또는 매핑을 이용해서 데이터 변경하기 
#### DataFrame의 컬럼이나, Series, 배열 내의 값을 기반으로 데이터의 형태를 변화하고자 할때.

In [60]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                             'Pastrami', 'corned beef', 'Bacon',
                            'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [61]:
#해당 육류가 어떤 동물의 고기인지 알려주는 컬럼을 추가한다고 가정하자.
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

In [62]:
lowercased = data['food'].str.lower() #데이터에 대소문자 섞여있으므로, 소문자로 변경
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [63]:
#Series의 map 메소드는 사전류의 객체나 어떤 함수를 받을 수 있다.
data['animal'] = lowercased.map(meat_to_animal) #map 메소드 이용
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [64]:
data['food'].map(lambda x: meat_to_animal[x.lower()]) #함수를 넘겨서 같은 일을 수행 가능

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 값 치환하기 
#### fillna같이 누락된 값을 채우는 일은 일반적인 치환 작업이다. 
#### 위에서 map 메소드를 한 객체안에서 값의 부분집합을 변경하는데 사용하였다.
#### replace 메소드는 같은 작업에 대해 보다 간단하고 유연한 방법을 제공한다. 

In [65]:
data = pd.Series([1., -999., 2., -999., 3.])
data

0      1.0
1   -999.0
2      2.0
3   -999.0
4      3.0
dtype: float64

In [66]:
data.replace(-999, np.nan) #replace를 이용한 치환

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

In [67]:
data.replace([-999, 1.0], np.nan) #여러개의 값을 치환하려면 값의 리스트를 넘기면 된다.

0    NaN
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

In [68]:
data.replace([-999, 1.0], [np.nan, 0]) #치환하려는 값마다 다른 값으로 치환하려면 새로 지정할 리스트를 넘긴다.

0    0.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

In [69]:
data.replace({-999: np.nan, -1000: 0}) #리스트 대신 사전을 사용하는 것도 가능하다

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

### 축 색인 이름 바꾸기 
#### Series의 값들처럼 축 이름 역시 유사한 방식으로 바꿀 수 있다. 

In [84]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                   index = ['Ohio', 'Colorado', 'New York'],
                   columns = ['one', 'two', 'three', 'four'])

In [85]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [86]:
transform = lambda x: x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [87]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [88]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [91]:
data.rename(index=str.title, columns=str.upper) #원래 객체를 변경하지 않고 새로운 객체를 생성하려면 rename 메소드를 사용한다.
#Series.str.lower : Converts all characters to lowercase.
#Series.str.upper : Converts all characters to uppercase.
#Series.str.title : Converts first character of each word to uppercase and remaining to lowercase.
#Series.str.capitalize :Converts first character to uppercase and remaining to lowercase.
#Series.str.swapcase :Converts uppercase to lowercase and lowercase to uppercase.

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [92]:
data.rename(index={'OHIO':"INDIANA"},
           columns = {'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [93]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [94]:
#rename 메소드를 사용하면 DataFrame을 직접 복사해서 index와 column 속성을 갱신할 필요없이 바로 변경 가능
#원본 데이터를 바로 변경하려면, inplace=True 옵션을 사용하면 된다.
data.rename(index={'OHIO': 'INDIANA'}, inplace=True) 
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [95]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [80]:
import pandas as pd
import numpy as np

### 개별화와 양자화 
#### 연속성 데이터는 종종 개별로 분할 하거나 아니면 분석을 위해 그룹별로 나누기도 한다. 

In [11]:
#수업에 참여하는 학생 그룹 데이터가 있고, 나이에 따라 분류한다고 가정하자.
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]cccccccc
bins = [18, 25, 35, 60, 100]

cats = pd.cut(ages, bins) #cut 함수를 이용해서 18-25, 26-35, 36-60, 61-100 그룹으로 나눈다.
cats
#pandas에서 반환하는 객체는 Categorical이라는 특수한 객체이다. 이 객체는 그룹 이름이 담긴 배열이라고 생각하면 된다. 

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [12]:
cats.codes  #4개의 categorie중 어디에 속하는지 표시

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [98]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [13]:
cats.categories[0]

Interval(18, 25, closed='right')

In [99]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

#### 간격을 나타내는 표기법은 중괄로 시작해서 대괄호로 끝난다. 
#### 중괄호 쪽의 값은 포함하지 않고 대괄호쪽의 값은 포함한다. 
#### right=Flase 를 사용해서 중괄호 대신 대괄호쪽이 포함되지 않게 바꿀 수 있다.

In [100]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [101]:
#labels 옵션을 사용해서 그룹의 이름을 직접 넘겨줄 수도 있다. 
group_names = ['Youth', 'YoungAdult', 'MiddleAges', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAges', 'MiddleAges', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAges' < 'Senior']

#### 만약 cut함수에 명시적으로 그룹의 경계값을 넘기지 않고 그룹의 갯수를 넘겨주면, 데이터의 최소값과 최대값을 기준으로 자동으로 균등한 길이의 그룹을 자동 생성한다. 

In [102]:
#어떤 균등분포에서 4개의 그룹으로 나누는 경우
data = np.random.rand(20)
data

array([0.93554391, 0.08280299, 0.59611316, 0.36341499, 0.61681015,
       0.67764284, 0.36788773, 0.86623073, 0.01511093, 0.07601995,
       0.94919668, 0.70476268, 0.95584376, 0.11544219, 0.21484838,
       0.14997017, 0.765457  , 0.99953022, 0.39120459, 0.55388421])

In [103]:
pd.cut(data, 4, precision=2) #precision=2 옵션은 소수점 아래 2자리까지 제한한다.

[(0.75, 1.0], (0.014, 0.26], (0.51, 0.75], (0.26, 0.51], (0.51, 0.75], ..., (0.014, 0.26], (0.75, 1.0], (0.75, 1.0], (0.26, 0.51], (0.51, 0.75]]
Length: 20
Categories (4, interval[float64]): [(0.014, 0.26] < (0.26, 0.51] < (0.51, 0.75] < (0.75, 1.0]]

In [104]:
data = pd.cut(data, 4, precision=2) 
pd.value_counts(data)

(0.75, 1.0]      6
(0.014, 0.26]    6
(0.51, 0.75]     5
(0.26, 0.51]     3
dtype: int64

#### cut 함수와 유사한 qcut함수가 있다. 
#### qcut함수는 표본 변위치를 기준으로 나눈다. cut함수를 사용하면 데이터의 분산에 따라 각각의 그룹마다 데이터 수가 다르게 나누어지는데,
#### qcut함수는 표준 변위치를 사용하기 때문에 적당히 같은 크기의 그룹으로 나눌 수 있다.

In [109]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
cats

[(-3.29, -0.672], (0.0337, 0.677], (0.0337, 0.677], (0.677, 3.579], (0.677, 3.579], ..., (-3.29, -0.672], (-0.672, 0.0337], (0.677, 3.579], (-3.29, -0.672], (0.677, 3.579]]
Length: 1000
Categories (4, interval[float64]): [(-3.29, -0.672] < (-0.672, 0.0337] < (0.0337, 0.677] < (0.677, 3.579]]

In [110]:
pd.value_counts(cats)

(0.677, 3.579]      250
(0.0337, 0.677]     250
(-0.672, 0.0337]    250
(-3.29, -0.672]     250
dtype: int64

In [111]:
cats = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]) #cut함수와 유사하게 변위치를 직접지정 가능(변위치는 0에서 1까지)

In [112]:
pd.value_counts(cats )

(0.0337, 1.259]     400
(-1.249, 0.0337]    400
(1.259, 3.579]      100
(-3.29, -1.249]     100
dtype: int64

### 특이값(outlier) 검출 및 제외 

In [14]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.010928,-0.018441,0.049777,-0.002386
std,1.019481,0.983585,0.932966,1.026008
min,-3.257589,-3.050411,-3.376689,-3.521655
25%,-0.678667,-0.688367,-0.616909,-0.691318
50%,-0.005533,-0.001227,0.053034,-0.001261
75%,0.659099,0.619380,0.682072,0.715220
max,2.943624,3.064708,2.914365,3.149239


In [15]:
data

,0,1,2,3
0,-2.023966,1.612986,-1.084061,-0.934566
1,-0.807875,0.759997,-0.942991,0.512842
2,-2.322598,-2.753747,-0.892539,-0.280717
3,0.784232,-0.341663,-0.396669,0.237285
4,-1.328681,0.087664,0.801138,0.075143
...,...,...,...,...
995,-1.875832,-0.313346,0.995286,-1.825983
996,-0.879110,1.044428,-0.574067,-0.767135
997,0.791657,0.903542,-0.270068,-1.871425
998,2.561540,-0.123745,0.186450,0.119137


In [19]:
col = data[2]
col[col.abs() > 3] #DataFrame의 특정 한 컬럼(data[2])에서 3을 초과하는 값을 찾는 경우

195   -3.376689
Name: 2, dtype: float64

In [23]:
data[(np.abs(data) > 3).any(axis='columns')] #3을 초과하는 모든 row를 선택하려면 any 메소드를 사용

,0,1,2,3
133,0.191165,-3.050411,-0.307586,-1.674989
195,-1.076821,-1.065954,-3.376689,0.367066
322,-3.257589,-1.223690,-0.229803,-0.921911
547,0.400203,0.242759,0.578589,3.149239
621,1.605794,3.064708,0.004156,-0.183668
816,-1.189717,-0.071689,-0.702974,-3.521655


In [24]:
# Capping outside -3 to 3
data[np.abs(data) > 3] = np.sign(data) * 3  #-3이나 3을 초과하는 값을 -3 또는 3으로 지정, np.sign(data)는 부호에 따라 1이나 -1을 반환
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.010671,-0.018455,0.050154,-0.002013
std,1.018692,0.983230,0.931656,1.023900
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.678667,-0.688367,-0.616909,-0.691318
50%,-0.005533,-0.001227,0.053034,-0.001261
75%,0.659099,0.619380,0.682072,0.715220
max,2.943624,3.000000,2.914365,3.000000


In [25]:
data.head() #head는 기본으로 5개 출력, head(10)이면 10개 출력

,0,1,2,3
0,-2.023966,1.612986,-1.084061,-0.934566
1,-0.807875,0.759997,-0.942991,0.512842
2,-2.322598,-2.753747,-0.892539,-0.280717
3,0.784232,-0.341663,-0.396669,0.237285
4,-1.328681,0.087664,0.801138,0.075143


In [26]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,-1.0
1,-1.0,1.0,-1.0,1.0
2,-1.0,-1.0,-1.0,-1.0
3,1.0,-1.0,-1.0,1.0
4,-1.0,1.0,1.0,1.0


### 치환(permutation)과 임의 샘플링 
#### nimpy.random.permutation함수를 사용하면 row를 쉽게 임의 순서로 재배치 할 수있다.

In [37]:
df = pd.DataFrame(np.arange(5*7).reshape((5,7)))
df
sampler = np.random.permutation(5)
sampler

array([0, 3, 2, 1, 4])

In [38]:
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [41]:
df.take(sampler) #take 함수를 사용, take를 호출할때, axis='columns'를 넘기면 열에 대해 작동한다.
#df.take(sampler, axis='columns')

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34


In [32]:
df.iloc[sampler] #iloc 색인으로 사용

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6


In [35]:
column_sampler = np.random.permutation(7)
column_sampler
df.take(column_sampler, axis="columns")

,6,4,5,3,0,1,2
0,6,4,5,3,0,1,2
1,13,11,12,10,7,8,9
2,20,18,19,17,14,15,16
3,27,25,26,24,21,22,23
4,34,32,33,31,28,29,30


In [33]:
df.sample(n=3) #치환 없이 일부만 임의로 선택하려면 sample 메소드 사용

,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6


In [34]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True) #반복선택을 허용하며, 표본을 치환을 통해 생성해내려면 replace=True 옵션 사용
draws

2   -1
2   -1
2   -1
4    4
3    6
1    7
0    5
3    6
0    5
1    7
dtype: int64

### 표시자/더미(dummy) 변수 생성하기 
#### 통계나 머신러닝 에플리케이션을 위해 분류값을 '더미(dummy)'나 '표시자(indicator)' 행렬로 전환한다. 
#### 한 컬럼에 k가지의 값이 있다면, k개의 컬럼이 있는 행렬을 만들고 값으로는 1과 0을 채워 넣는다. 

In [42]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                  'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [43]:
pd.get_dummies(df['key'], dtype=float )

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [44]:
dummies = pd.get_dummies(df['key'], prefix='key', dtype=float) #컬럼에 접두어(prefix)를 추가한 후 다른 데이터와 병합, prefix인자 사용
dummies


,key_a,key_b,key_c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [45]:
df_with_dummy = df[['data1']].join(dummies) 
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


#### DataFrame의 한 row가 여러 카테고리에 속하다면 조금 복잡해진다. 
#### 'MovieLens의 영화 평점 데이터' 에서 살펴본다. 

In [46]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                      header=None, names=mnames, engine="python")
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [49]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


In [54]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic.iloc[0]
#movies_windic.iloc[0:2]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

#### get_dummies와 cut 같은 이산함수를 잘 조합하면 통계 애플리케이션에서 유용하게 사용할 수 있다. 

In [55]:
np.random.seed(12345)
values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [56]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.cut(values, bins)

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

## 문자열 다루기 

In [133]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


### 확장 데이터 유형 
#### pandas는 numpy 기반으로 만들어 졌다. numpy 기반으로 인해 미묘한 문제가 발생할 수 있다. 
#### 이를 위해 pandas "확장 데이터 유형"을 제공한다.  
#### 표 7-3 참조

In [57]:
s = pd.Series([1, 2, 3, None])
s
s.dtype

dtype('float64')

In [58]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype()) #Int64Dtype 이라는 확장 데이터 유형 사용
s
s.isna()
s.dtype

Int64Dtype()

In [59]:
s[3]
s[3] is pd.NA

True

In [61]:
s = pd.Series([1, 2, 3, None], dtype="Int64") #Int64Dtype 대신에 Int64 사용가능(대문자 주의) 
s.dtype

Int64Dtype()

In [62]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

0      one
1      two
2     <NA>
3    three
dtype: string

In [63]:
df = pd.DataFrame({"A": [1, 2, None, 4],
                   "B": ["one", "two", "three", None],
                   "C": [False, None, False, True]})
df
df["A"] = df["A"].astype("Int64")
df["B"] = df["B"].astype("string")
df["C"] = df["C"].astype("boolean")
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


### 문자열 객체 메소드 
#### 문자열을 다루어야 하는 대부분의 에플리케이션은 내장 문자열 메소드만으로도 충분하다. 

In [64]:
val = 'a,b,   guido'
val.split(',')  #,로 구분된 문자열은 split 메소드를 이용해 분리

['a', 'b', '   guido']

#### 양 끝 공백 제거 : .strip()
#### 왼쪽 공백 제거 : .lstrip()
#### 오른쪽 공백 제거 : .rstrip() 

In [66]:
pieces = [x.strip() for x in val.split(',')] #split은 종종 공백문자(줄바꿈 문자 포함)를 제거하는 strip과 조합해서 사용한다.
pieces  #출력결과 공백이 제거된것을 확인

['a', 'b', 'guido']

In [67]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [68]:
'::'.join(pieces) #위의 방법보다 편리하고 파이썬스러운 방법, 리스트나 튜플을 ::문자열의 join메소드로 전달하는것이다.

'a::b::guido'

#### 일치하는 부분문자열의 위치를 찾는 방법도 있다. index나 find를 사용하는것도 가능하지만 
#### 파이썬의 in 예약어를 사용하면 일치하는 문자열을 쉽게 찾을 수 있다. 

In [69]:
'guido' in val

True

In [70]:
val.index(',') #index의 경우 찾으면 1을 반환

1

In [71]:
val.find(':') #find의 경우 찾지 못하면 -1을 반환

-1

In [72]:
val.index(':') #index의 경우 찾지 못하면 예외를 발생한다. index와 find의 차이점은 index의경우 찾지 못하면 예외를 발생한다.

ValueError: substring not found

In [73]:
val.count(',')

2

In [74]:
val.replace(',', '::')  #replace는 찾는 퍄턴을 다른 문자열로 치환한다.

'a::b::   guido'

In [75]:
val.replace(',', '')

'ab   guido'

### 정규 표현식
#### 파이썬에는 re 모듈이 내장되어 있어 문자열에 대한 정규 표현식을 처리한다. 

In [3]:
import re

text = "foo bar\t baz   \tqux" #여러가지 공백문자(탭, 스페이스, 개행문자)가 포함된 문자열
text

'foo bar\t baz   \tqux'

In [4]:
re.split('\s+', text)   #'\s+'는 하나 이상의 공백문자를 의미한다.
#정규 표현식이 먼저 컴파일되고, 그 다음 split 메서드가 실행된다.

['foo', 'bar', 'baz', 'qux']

In [5]:
regex = re.compile('\s+') #먼저 컴파일
regex.split(text)         #정규표현식 객체 재사용

['foo', 'bar', 'baz', 'qux']

In [6]:
regex.findall(text) #정규 표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall 메소드 사용

[' ', '\t ', '   \t']

#### match와 search는 findall 메서드와 관련이 있다.
#### findall : 문자열에서 일치하는 모든 부분 문자열을 찾아준다. 
#### search : 패턴과 일치하는 첫 번째 존재를 반환한다. 
#### match : 문자열의 시작부분에서 일치하는 것만 찾아준다 
#### sub : 찾은 패턴을 주어진 문자열로 치환하여 새로운 문자열을 반환

In [7]:
#이메일 주소를 검사하는 정규 표현식
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
jwseok@changwon.ac.kr
"""

pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"
regex = re.compile(pattern, flags=re.IGNORECASE) #re.IGNORECASE는 정규 표현식이 대소문자를 가리지 않도록 한다.

In [8]:
regex.findall(text) #이메일 주소와 일치하는 모든 부분 문자열을 찾아준다

['dave@google.com',
 'steve@gmail.com',
 'rob@gmail.com',
 'ryan@yahoo.com',
 'jwseok@changwon.ac.kr']

In [13]:
m = regex.search(text)  #패턴과 일치하는 첫번째 존재
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [14]:
text[m.start():m.end()]

'dave@google.com'

In [11]:
print(regex.match(text)) #문자열의 시작부분에서 일치하는 것

None


In [19]:
r = re.compile("[ab]")
print(r.search("pizza")) #패턴과 일치하는 첫번째 존재
print(r.match("pizza"))  #문자열의 시작부분에서 일치하는 것
print(r.match("abcd"))

<re.Match object; span=(4, 5), match='a'>
None
<re.Match object; span=(0, 1), match='a'>


In [20]:
print(regex.sub('REDACTED', text)) #sub 메소드는 찾은 패턴을 주어진 문자열로 치환하여 새로운 문자열을 반환한다.

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED
REDACTED



In [21]:
#이메일 주소를 각 콤포논트별(사용자이름, 도메인이름, 도메인접미사)로 나누어야 한다면, 각 패턴을 괄호로 묶어준다.
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})' 
regex = re.compile(pattern, flags=re.IGNORECASE)

In [22]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com'),
 ('jwseok', 'changwon.ac', 'kr')]

In [23]:
m = regex.match('wes@bright.net')
m.groups()  #group메서드로 각 패넌 컴포넌트의 튜플을 얻을 수 있다.

('wes', 'bright', 'net')

In [156]:
#sub 은 \1, \2와 같은 특수한 기호를 사용하여 각 패턴그룹에 접근할 수 있다.
#\1은 첫 번째, \2는 두 번째 그룹을 의미한다.
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com
Username: jwseok, Domain: changwon.ac, Suffix: kr



### 벡터화된 문자열 함수

In [24]:
import pandas as pd
import numpy as np
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [25]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

#### 문자열고 정규표현식 메서는 data.map을 사용
#### 하지만, NA 값을 만나면 실패하게 된다. 
#### 이 경우 Series에서는 str속성을 이용해서 처리한다. 
#### 정규표현식을 'IGNORECASE' 같은 re 옵션과 함께 사용하는 것도 가능하다. 

In [31]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [32]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [33]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

#### To retrieve elements from Vector, we can either use 'str.get' or 'index'.
#### To access elements in embedded list, we pass an index to either of these functions.
#### We can also slice strings with this syntax.

In [35]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [169]:
data.str[:5]  #문자열을 잘라낼 수 있다.

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

### 범주형 데이터
#### 범주형(categorical)을 활용하여 메모리 사용량을 줄이고 성능 개선
#### 통계와 머신러닝에 유용한 도구로 활용

In [37]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [38]:
pd.unique(values)  #하나의 열에서 특정 값이 반복되는 경우, 유일값 추출

array(['apple', 'orange'], dtype=object)

In [39]:
pd.value_counts(values)#하나의 열에서 특정 값이 반복되는 경우, 반복되는 횟수 추출

apple     6
orange    2
dtype: int64

In [40]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])

In [41]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [42]:
dim

0     apple
1    orange
dtype: object

In [43]:
dim.take(values) #take 메서드를 사용하면, Series에 저장된 원래 문자열을 구할 수 있다. 
# 이러한 표현을 범주형 표기법이라 한다.

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

#### 판다스의 Categorical 확장형
##### 판다스에서는 정수 기반의 범주형 데이터를 표현하는 categorical 확장형이 있다.
##### 유사한 값이 다수 존재하는 경우, 데이터를 효율적으로 압축하여 적은 메모리에서 바른 성능을 낸다.

In [45]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
rng = np.random.default_rng(seed=12345)
#NumPy 버전 1.17부터 추가된 새로운 난수 생성기(Generator)이다. random.seed에 비해 다양한 난수분포를 가질 수 있고,
#여러개의 난수를 관리하는 등 강력한 기능을 제공
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': rng.integers(3, 15, size=N),
                   'weight': rng.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [46]:
fruit_cat = df['fruit'].astype('category') #범주형 데이터로 변경
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [47]:
c = fruit_cat.array #.array 속성으로 접근, fruit_cat은 pandas.Categorical 인스턴스이다.
type(c)
#Categorical객체에는 categories와 code 속성이 있다.

pandas.core.arrays.categorical.Categorical

In [50]:
c.categories  #categories 속성

Index(['apple', 'orange'], dtype='object')

In [51]:
c.codes      # code 속성

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [52]:
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [53]:
df['fruit'] = df['fruit'].astype('category')  #DataFrame의 열을 범주형으로 변경
df["fruit"]

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [59]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar']) #파이썬 시퀀스에서 pandas.Categorical을 직접 생성
my_categories
#my_categories.codes
#my_categories.categories

Index(['bar', 'baz', 'foo'], dtype='object')

In [55]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)  #기존에 정의된 categories와 code가 있다면 from_codes로 생성
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [61]:
# 범주형 변경시 순서를 보장하지 않는다. -> 순서를 지정할 수 있다.
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [63]:
my_cats_2.as_ordered()  #순서가 없는 범주현 인스턴스는 as_ordered 매서드사용해서 정렬

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

#### Categorical 연산

In [64]:
rng = np.random.default_rng(seed=12345)
draws = rng.standard_normal(1000)
draws[:5]

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

In [65]:
bins = pd.qcut(draws, 4)
bins

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] < (0.687, 3.211]]

In [73]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins
bins.codes[:10]

array([0, 3, 0, 1, 1, 0, 0, 2, 2, 0], dtype=int8)

In [74]:
bins = pd.Series(bins, name='quartile')
bins

0      Q1
1      Q4
2      Q1
3      Q2
4      Q2
       ..
995    Q3
996    Q3
997    Q2
998    Q3
999    Q2
Name: quartile, Length: 1000, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [75]:
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


In [76]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

#### 범주형을 활용한 성능 개선

In [77]:
N = 10_000_000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))  #천만개의 Series

In [78]:
labels

0          foo
1          bar
2          baz
3          qux
4          foo
          ... 
9999995    qux
9999996    foo
9999997    bar
9999998    baz
9999999    qux
Length: 10000000, dtype: object

In [79]:
categories = labels.astype('category')

In [82]:
labels.memory_usage(deep=True)

600000128

In [83]:
categories.memory_usage(deep=True)

10000528

In [84]:
%time _ = labels.astype('category')

Wall time: 461 ms


In [85]:
%timeit labels.value_counts()

658 ms ± 5.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [86]:
%timeit categories.value_counts()

65.7 ms ± 499 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Categorical 매서드

In [87]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [89]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [90]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [91]:
# 데이터의 실제 범주가 네 종류를 넘는다면, set_categories먀서드 이용 변경
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories) 
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [93]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [94]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [95]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [96]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

#### 모델링을 위한 더미 변수 생성하기

In [98]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [99]:
pd.get_dummies(cat_s, dtype=float)

,a,b,c,d
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,1.0
